In [1]:
import os
host = os.getenv('sqlHOST')
username = os.getenv('sqlUSER')
password = os.getenv('sqlPSWD')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from env import get_db_url
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")
import wrangle

**INITIAL HYPOTHESIS**

**Payment type, contract type and internet service type are all major drivers of customer churn at Telco.**

In [2]:
# ACQUIRE DATA
df = pd.read_csv('telco.csv')

In [19]:
# PREPARE DATA
clean_telco = wrangle.prep_telco_data(df)
clean_telco.head()

,contract_type_Month-to-month,contract_type_One year,contract_type_Two year,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check,churn,contract_type,internet_service_type,payment_type
0,0,1,0,1,0,0,0,0,0,1,No,One year,DSL,Mailed check
1,1,0,0,1,0,0,0,0,0,1,No,Month-to-month,DSL,Mailed check
2,1,0,0,0,1,0,0,0,1,0,Yes,Month-to-month,Fiber optic,Electronic check
3,1,0,0,0,1,0,0,0,1,0,Yes,Month-to-month,Fiber optic,Electronic check
4,1,0,0,0,1,0,0,0,0,1,Yes,Month-to-month,Fiber optic,Mailed check


In [4]:
# SPLIT DATA
train, test = train_test_split(clean_telco, test_size=.2, random_state=123)
train, validate = train_test_split(train, test_size=.3, random_state=123)
print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

train -> (3943, 14)
validate -> (1691, 14)
test -> (1409, 14)


**KEY QUESTIONS**

**Which payment types are most associated with customer churn?**   
    -- electronic check  
    -- mailed check  
    -- bank transfer   
    -- credit card    
**Which contract types are most associated with customer churn?**  
    -- month-to-month  
    -- 1-year  
    -- 2-year  
**Which internet service types are most associated with customer churn?**  
    -- fiber-=optic  
    -- DSL  
    -- none  

In [5]:
# EXPLORE PAYMENT TYPE DATA
# Null hypothesis: Payment type has no impact on churn likelihood.
# Alternative hypothesis: Payment type does have an impact on churn likelihood.
payment_type = pd.crosstab(train.churn, train.payment_type) 
alpha = 0.05
chi2, p, degf, expected = stats.chi2_contingency(payment_type)
print('Observed Values\n')
print(payment_type.values)
print('---\nExpected Values\n')
print(payment_type.astype(int))
print('---\n')
print(f'chi^2 = {chi2:.4f}') 
print(f'p     = {p:.4f}')
print('---\n')
if p < alpha:
    print('CONCLUSION: Payment type does have an impact on churn likelihood.')
else:
    print('CONCLUSION: Payment type has no impact on churn likelihood.')

Observed Values

[[734 725 672 762]
 [146 130 604 170]]
---
Expected Values

payment_type  Bank transfer (automatic)  Credit card (automatic)  \
churn                                                              
No                                  734                      725   
Yes                                 146                      130   

payment_type  Electronic check  Mailed check  
churn                                         
No                         672           762  
Yes                        604           170  
---

chi^2 = 416.0771
p     = 0.0000
---

CONCLUSION: Payment type does have an impact on churn likelihood.


In [6]:
# EXPLORE CONTRACT TYPE DATA
# Null hypothesis: Contract type has no impact on churn likelihood.
# Alternative hypothesis: Contract type does have an impact on churn likelihood.
contract_type = pd.crosstab(train.churn, train.contract_type) 
alpha = 0.05
chi2, p, degf, expected = stats.chi2_contingency(contract_type)
print('Observed Values\n')
print(contract_type.values)
print('---\nExpected Values\n')
print(contract_type.astype(int))
print('---\n')
print(f'chi^2 = {chi2:.4f}') 
print(f'p     = {p:.4f}')
print('---\n')
if p < alpha:
    print('CONCLUSION: Contract type does have an impact on churn likelihood.')
else:
    print('CONCLUSION: Contract type has no impact on churn likelihood.')

Observed Values

[[1225  740  928]
 [ 931   86   33]]
---
Expected Values

contract_type  Month-to-month  One year  Two year
churn                                            
No                       1225       740       928
Yes                       931        86        33
---

chi^2 = 678.1617
p     = 0.0000
---

CONCLUSION: Contract type does have an impact on churn likelihood.


In [7]:
# EXPLORE INTERNET SERVICE TYPE DATA
# Null hypothesis: Internet service type has no impact on churn likelihood.
# Alternative hypothesis: Internet service type does have an impact on churn likelihood.
internet_service_type = pd.crosstab(train.churn, train.internet_service_type) 
alpha = 0.05
chi2, p, degf, expected = stats.chi2_contingency(internet_service_type)
print('Observed Values\n')
print(internet_service_type.values)
print('---\nExpected Values\n')
print(internet_service_type.astype(int))
print('---\n')
print(f'chi^2 = {chi2:.4f}') 
print(f'p     = {p:.4f}')
print('---\n')
if p < alpha:
    print('CONCLUSION: Internet service type does have an impact on churn likelihood.')
else:
    print('CONCLUSION: Internet service type has no impact on churn likelihood.')

Observed Values

[[1102  993  798]
 [ 259  727   64]]
---
Expected Values

internet_service_type   DSL  Fiber optic  None
churn                                         
No                     1102          993   798
Yes                     259          727    64
---

chi^2 = 418.2304
p     = 0.0000
---

CONCLUSION: Internet service type does have an impact on churn likelihood.


**KEY FINDING 1**

**-- Payment type, contract type and internet service type are confirmed to be drivers of customer churn at Telco.**

In [8]:
# BASELINE ACCURACY

In [9]:
baseline_accuracy = (train.churn == 'No').mean()
baseline_accuracy
print(f'BASELINE ACCURACY = {baseline_accuracy:.4f}') 

BASELINE ACCURACY = 0.7337


In [10]:
# DECISION TREE MODELING

In [11]:
X_train = train.drop(columns=['churn','contract_type',
       'internet_service_type','payment_type'])
X_validate = validate.drop(columns=['churn', 'contract_type',
       'internet_service_type', 'payment_type'])
X_test = test.drop(columns=['churn', 'contract_type',
       'internet_service_type', 'payment_type'])

In [12]:
y_train = train.churn
y_validate = validate.churn
y_test = test.churn
clf = DecisionTreeClassifier(max_depth=3, random_state=123)
clf = clf.fit(X_train, y_train)
clf.score(X_train, y_train)

0.7742835404514329

**KEY FINDING 2**

**-- Decision tree model for predicting churn generates a mean accuracy score of 77.4%, above the baseline accuracy of 73.4% in our train data set.**

In [ ]:
# RANDOM FOREST MODELING

In [21]:
# Make the model
forest1 = RandomForestClassifier(max_depth=1, random_state=123)

# Fit the model (on train and only train)
forest1.fit(X_train, y_train)

# Use the model
# We'll evaluate the model's performance on train, first
y_predictions = forest1.predict(X_train)

# Produce the classification report on the actual y values and this model's predicted y values
report = classification_report(y_train, y_predictions, output_dict=True)
print("Tree of 1 depth")
pd.DataFrame(report)

Tree of 1 depth


,No,Yes,accuracy,macro avg,weighted avg
precision,0.733705,0.0,0.733705,0.366853,0.538323
recall,1.000000,0.0,0.733705,0.500000,0.733705
f1-score,0.846401,0.0,0.733705,0.423201,0.621009
support,2893.000000,1050.0,0.733705,3943.000000,3943.000000
